In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from utils.diff import diff

In [4]:
# 64.  rd
# 季度频率。当研发费用占总资产的比例增加大于5%时，指标变量为1。
df1 = pd.read_csv('季_资产负债表.csv', usecols=['证券代码', '会计期间', '资产总计'], low_memory=False)
# 确保证券代码为整数类型
df1['证券代码'] = pd.to_numeric(df1['证券代码'], errors='coerce').fillna(0).astype(int)

df2 = pd.read_csv('季_利润表.csv', usecols=['证券代码', '会计期间', '研发费用', '营业收入', '查询成功'])
# 确保证券代码为整数类型
df2['证券代码'] = pd.to_numeric(df2['证券代码'], errors='coerce').fillna(0).astype(int)

df2['研发费用'][df2['查询成功'].astype('bool')] = df2['研发费用'][df2['查询成功'].astype('bool')].fillna(0)
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')
df3 = diff(df3, col=['研发费用', '营业收入'])
df3['rd1'] = df3['研发费用']/df3['资产总计'].replace(0, np.nan)
df3['rd2'] = (df3['rd1'] - df3.groupby('证券代码').shift(1)['rd1'])/df3.groupby('证券代码').shift(1)['rd1']
df3['rd'] = np.where(df3['rd2'] > 0.05, 1, 0)
df3['rd'][df3['rd2'].isna()] = np.nan
df3['rd'][(df3.groupby('证券代码').shift(1)['rd1'] == 0) & (df3['rd1'] == 0)] = 0
df3['rd'][(df3.groupby('证券代码').shift(1)['rd1'] == 0) & (df3['rd1'] > 0)] = 1

# 65.  rd_mve
# 季度频率。研发费用除以季度末市值。
df4 = pd.read_csv('季_相对价值指标.csv', usecols=['证券代码', '会计期间', '市值A'])
# 确保证券代码为整数类型
df4['证券代码'] = pd.to_numeric(df4['证券代码'], errors='coerce').fillna(0).astype(int)
df5 = pd.merge(df3, df4, on=['证券代码', '会计期间'], how='left')
df5['rd_mve'] = df5['研发费用']/df5['市值A'].replace(0, np.nan)

# 66.  rd_sale
# 季度频率。研发费用除以季度销售额。
df5['rd_sale'] = df5['研发费用']/df5['营业收入'].replace(0, np.nan)

df6 = df5[['证券代码', '会计期间', 'rd', 'rd_mve', 'rd_sale']]
df6.to_csv('季_64_65_66.csv', index=False, encoding='utf-8-sig')
df6

,证券代码,会计期间,rd,rd_mve,rd_sale
0,1,2000-03-31,NaN,NaN,NaN
1,1,2000-06-30,NaN,NaN,NaN
2,1,2000-09-30,NaN,NaN,NaN
3,1,2000-12-31,NaN,NaN,NaN
4,1,2001-03-31,NaN,NaN,NaN
...,...,...,...,...,...
571126,920118,2023-09-30,NaN,NaN,NaN
571127,920118,2023-12-31,NaN,NaN,NaN
571128,920118,2024-03-31,NaN,NaN,NaN
571129,920118,2024-06-30,NaN,NaN,NaN
